In [1]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.risk_models import sample_cov
from pypfopt.risk_models import fix_nonpositive_semidefinite

In [2]:
import numpy as np
import pandas as pd
import quandl
import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import shutil
import glob
import math
import requests
import zipfile
from bs4 import BeautifulSoup

In [3]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.efficient_frontier import EfficientFrontier

In [4]:
from datetime import datetime, timedelta

In [12]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.expected_returns import returns_from_prices

In [5]:
# uploading daily quotations of polish companies for the last 10y
def get_stock_data(path):
    url = ["https://info.bossa.pl/pub/ciagle/omega/omegacgl.zip", "https://info.bossa.pl/pub/newconnect/omega/omegancn.zip"]
    request = []
    for i in url:
        request.append(requests.get(i, allow_redirects=True))
    for i,j in enumerate(request):
        with open(path+str(i)+'Stocks.zip', 'wb') as f:
            f.write(j.content)
        with zipfile.ZipFile(path+str(i)+'Stocks.zip', 'r') as zip_ref:
            zip_ref.extractall(path+'Stocks/')
    read_files = glob.glob(path+'Stocks/*.txt')
    with open("result.txt", "wb") as outfile:
        for f in read_files:
            with open(f, "rb") as infile:
                outfile.write(infile.read())
    data = pd.read_table('result.txt', sep =',', header=0)
    data.columns = ['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']
    data = data.loc[data['Volume'] != 'Volume']
    data = data.loc[data['Volume'] != '<VOL>']
    convert_dtypes = {'Volume': float, 
                      'Close': float,
                      'Date':'datetime64', 
                      'Ticker':'string'}
    data = data.astype(convert_dtypes)
    data.drop(columns = ['Open', 'High', 'Low'], inplace = True)
    return data

In [6]:
path = 'C:/Users/pawel/PycharmProjects/quandl/'

In [7]:
data = get_stock_data(path)

In [8]:
def get_fundamentals():
    df_companies = pd.read_csv('Companies_gvkey.csv').dropna()
    df_fundamentals = pd.read_csv('Polish_Fundamentals_Annual.csv')
    df_fundamentals = pd.merge(df_fundamentals,df_companies, on = 'gvkey', how = 'inner')
    df_fundamentals.rename(columns = {'fyear':'year'}, inplace = True)
    return df_fundamentals

In [16]:
df_fundamentals = get_fundamentals()

In [18]:
sectors = pd.read_csv('sectors_names.csv')

In [19]:
def combined_market_fundamentals(df_fundamentals, data):
    df_gvkey = df_fundamentals[['gvkey', 'Ticker']].groupby(['gvkey'])[['Ticker']].last().reset_index()
    data_combined = pd.merge(data, df_gvkey, on = 'Ticker', how = 'inner')
    return data_combined

In [20]:
data_combined = combined_market_fundamentals(df_fundamentals, data)

In [21]:
# creating year column based on date column
data_combined['year'] = data_combined.Date.dt.year

In [22]:
# function returns mean historical return of polish stocks
# param year enables to set a year of analysis
def annualized_mean_returns(data_combined, year = None):
    data_returns = data_combined.copy(deep=False)
    if year != None:
        data_returns = data_combined.loc[data_combined.year == year]
    data_returns = data_returns.groupby(['Ticker', 'Date'])[['Close']].mean().unstack(level=0)
    data_returns = mean_historical_return(data_returns, returns_data=False, compounding=True, frequency=252)
    data_returns = pd.DataFrame(data_returns).reset_index().drop(columns = ['level_0']).rename(columns = {0:'Return'}).sort_values('Return',ascending = False)
    if year != None:
        data_returns['year'] = year
    return data_returns

In [23]:
data_returns = annualized_mean_returns(data_combined, year = 2019)

In [24]:
# function returns defined percentage of mean return distribution
# e.g. x = 0.9 means that presented companies returns' are higher than returns of 90% companies in sample
def returns_x_quantile(x, data_returns, upper = True):
    if upper == True:
        data_quantile = data_returns.loc[data_returns.Return >= data_returns.quantile(x)[0]]
    else:
        data_quantile = data_returns.loc[data_returns.Return <= data_returns.quantile(x)[0]]
    return data_quantile

In [25]:
# function returns annualized covariance matrix
def annualized_matrix(data_combined, data_returns):
    data_matrix = pd.merge(data_combined, data_returns, on = ('Ticker', 'year'), how = 'inner')
    annualized_cov_matrix = sample_cov(data_matrix.groupby(['Ticker', 'Date'])[['Close']].mean().unstack(level=0))
    annualized_cov_matrix = fix_nonpositive_semidefinite(annualized_cov_matrix)
    return annualized_cov_matrix

In [34]:
# function returns weights of portfolio, which maximizes Sharpe ratio
def sharpe_opt(data_returns, annualized_cov_matrix):
    ef = EfficientFrontier(data_returns.set_index('Ticker').Return, annualized_cov_matrix)
    weights = ef.max_sharpe()
    weights = pd.DataFrame.from_dict(weights, orient = 'index').rename(columns = {0:'portfolio_share'}).sort_values('portfolio_share', ascending = False)
    return weights

In [ ]:
annualized_cov_matrix = annualized_matrix(data_combined, data_returns)

In [ ]:
weights = sharpe_opt(data_returns, annualized_cov_matrix)

In [ ]:
#sorting weights in dict
# weights = {k: v for k, v in sorted(weights.items(), key=lambda item: item[1], reverse = True)}

In [ ]:
weights.reset_index(inplace = True)

In [ ]:
weights.rename(columns = {'index':'Ticker'},inplace = True)

In [ ]:
def best_perform_stocks(data_combined, quantile, year = None):
    returns_t_0 = annualized_mean_returns(data_combined, year = year)
    returns_t_0 = returns_x_quantile(quantile, returns_t_0)
    returns_t_1 = annualized_mean_returns(data_combined, year = year+1)
    return_comp = pd.merge(returns_t_1, returns_t_0, on = 'Ticker', how = 'inner')
    return return_comp

In [ ]:
def worst_perform_stocks(data_combined, quantile, year = None):
    returns_t_0 = annualized_mean_returns(data_combined, year = year)
    returns_t_0 = returns_x_quantile(quantile, returns_t_0, upper = False)
    returns_t_1 = annualized_mean_returns(data_combined, year = year+1)
    return_comp = pd.merge(returns_t_1, returns_t_0, on = 'Ticker', how = 'inner')
    return return_comp

In [ ]:
# def trend_reversion(data_combined, percentile, lag_days, freq = 'W', drop = False):
    
#     prices_df = data_combined.groupby(['Ticker', 'Date'])[['Close']].mean().unstack(level=0)
#     prices_freq = returnd_df.groupby(pd.Grouper(level='Date', freq=freq)).mean()
#     prices_freq = returns_from_prices(prices_freq, log_returns=True)
#     prices_freq = prices_freq.stack(1)[['Close']]
    
#     if drop == False:
#         prices_freq = prices_freq.loc[prices_freq.Close > prices_freq.Close.quantile(percentile)]
#     else:
#         prices_freq = prices_freq.loc[prices_freq.Close < prices_freq.Close.quantile(percentile)]
        
#     prices_freq.reset_index(inplace=True)
#     df_lagged_combined = data_combined[0:0]
#     for i in range(len(prices_freq)):
#         df_lagged = data_combined.loc[(data_combined['Ticker'] == prices_freq.iloc[i,:][1]) & 
#                                       (data_combined['Date'] >= prices_freq.iloc[i,:][0]) & 
#                                       (data_combined['Date'] <= str(datetime.strptime(str(prices_freq.iloc[i,:][0]), '%Y-%m-%d %H:%M:%S')+timedelta(days=lag_days)))]
#         df_lagged_combined = pd.concat([df_lagged_combined,df_lagged])
        
#     df_lagged_combined = df_lagged_combined.groupby(['Ticker', 'Date'])[['Close']].mean().unstack(level=0)
#     df_lagged_combined = returns_from_prices(df_lagged_combined, log_returns=True)
#     return df_lagged_combined

### Fads hypothesis

In [41]:
# Negative long-term serial correlation in the performance of the aggregate market
# Function tests “fads hypothesis,” which asserts that the stock market might overreact to recent relevant news 

def reversal_effect(data_combined, percentile, lag_periods = 4, freq = 'W', drop = False):
    
    prices_df = data_combined.groupby(['Ticker', 'Date'])[['Close']].mean().unstack(level=0)
    prices_freq = prices_df.groupby(pd.Grouper(level='Date', freq=freq)).mean()
    returns_freq = returns_from_prices(prices_freq, log_returns=True)
    returns_freq = returns_freq.stack(1)[['Close']]
    
    if drop == False:
        returns_freq = returns_freq.loc[returns_freq.Close > returns_freq.Close.quantile(percentile)]
    else:
        returns_freq = returns_freq.loc[returns_freq.Close < returns_freq.Close.quantile(percentile)]
        
    prices_stacked = prices_freq.stack(1).rename(columns = {'Close':'Close_lag'})
    
    prices_combined = prices_stacked.join(returns_freq, on=['Date','Ticker'])
    prices_combined = prices_combined.reorder_levels(['Ticker', 'Date'])
    prices_combined['Close_lag_2'] = prices_combined.groupby(level=0)[['Close_lag']].shift(-lag_periods)
    prices_combined = prices_combined.loc[prices_combined.Close.notna()]
    prices_combined['ror_'+freq+str(lag_periods)] = np.log(prices_combined['Close_lag_2']/prices_combined['Close_lag'])
    prices_combined.dropna(subset=['ror_'+freq+str(lag_periods)], inplace=True)

    return prices_combined[['ror_'+freq+str(lag_periods)]]

In [42]:
prices_combined = reversal_effect(data_combined, 0.999, lag_periods = 21, freq = 'D')

In [45]:
prices_combined

ror_D21
Ticker    Date                
EFEKT     1993-08-05  1.044412
          1993-09-16  0.002497
BEST      2001-06-08  0.016086
NETIA     2001-10-10  0.501937
STALEXP   2001-11-19 -0.317302
...                        ...
REMEDIS   2020-11-19 -0.173847
INWESTPL  2020-11-20 -0.355034
OPENFIN   2020-11-20 -0.158748
ADVERTIGO 2020-11-23 -0.256296
APIS      2020-11-25 -0.474594

[2654 rows x 1 columns]

In [60]:
np.mean(prices_combined.ror_D21)

-0.13520668502161115

### Dividend yield hypothesis

In [46]:
# Function tests if the return on aggregate stock market tends to be higher when the dividend/price ratio, the dividend yield, is high

def dividend_price_ratio(data_combined, df_fundamentals, lag_years = 1, percentile = None):
    
    dv_df = pd.merge(data_combined, df_fundamentals[['gvkey', 'year','dv', 'cshoi']], on = ('gvkey', 'year'), how = 'inner').dropna(subset=['dv', 'cshoi'])
    dv_df['dv_p_share'] = dv_df['dv']/dv_df['cshoi']
    dv_df = dv_df.groupby(['Ticker', 'year'])[['Close', 'dv_p_share']].mean()
    dv_df['dv_yield'] = dv_df['dv_p_share']/dv_df['Close']
    
    df_price = data_combined.groupby(['Ticker','year'])[['Close']].last()
    df_returns = returns_from_prices(df_price.unstack(level=0), log_returns=True)
    df_returns = df_returns.stack(1).reorder_levels(['Ticker', 'year']).reset_index()
    df_returns = df_returns.groupby(['Ticker', 'year'])[['Close']].mean().shift(-lag_years)
    
    dv_df = dv_df[['dv_yield']].join(df_returns, on=['Ticker', 'year']).dropna()
    
    if percentile != None:
        dv_df_percentile = dv_df.loc[dv_df.dv_yield > dv_df.dv_yield.quantile(percentile)]
    else:
        dv_df_percentile = dv_df.copy(deep=False)
        
    dv_corr_returns = dv_df.dv_yield.corr(dv_df.Close)
    
    return np.mean(dv_df_percentile.Close), dv_corr_returns, dv_df_percentile

In [53]:
dv_mean, dv_corr, dv_df= dividend_price_ratio(data_combined, df_fundamentals, lag_years = 1, percentile = 0.9)

In [50]:
dv_mean

0.0992796991465278

In [51]:
dv_corr

0.07683296163258192

In [54]:
dv_df

dv_yield     Close
Ticker    year                    
4FUNMEDIA 2012  0.100787 -0.082521
          2013  0.123626 -1.148209
ACARTUS   2015  0.204943  0.087011
          2016  0.234432  0.117783
          2018  0.238589  0.075349
...                  ...       ...
XPLUS     2012  0.110694  0.087011
          2019  0.136680  1.007958
ZAMET     2012  0.182003  0.711839
ZEPAK     2018  0.120733  0.026317
ZYWIEC    2010  0.097408 -0.094427

[171 rows x 2 columns]

### Earnings yield hypothesis

In [55]:
# Function tests if the return on aggregate stock market tends to be higher when the earnings yield is high

def earnings_yield(data_combined, df_fundamentals, percentile = None):
    
    ib_df = pd.merge(data_combined, df_fundamentals[['gvkey', 'year','ib', 'cshoi']], on = ('gvkey', 'year'), how = 'inner').dropna(subset=['ib', 'cshoi'])
    ib_df['ib_p_share'] = ib_df['ib']/ib_df['cshoi']
    ib_df = ib_df.groupby(['Ticker', 'year'])[['Close', 'ib_p_share']].last()
    ib_df['ib_yield'] = ib_df['ib_p_share']/ib_df['Close']
    
    df_price = data_combined.groupby(['Ticker','year'])[['Close']].last()
    df_returns = returns_from_prices(df_price.unstack(level=0), log_returns=True)
    df_returns = df_returns.stack(1).reorder_levels(['Ticker', 'year']).reset_index()
    df_returns = df_returns.groupby(['Ticker', 'year'])[['Close']].mean().shift(-1)
    
    ib_df = ib_df[['ib_yield']].join(df_returns, on=['Ticker', 'year']).dropna()
    
    if percentile != None:
        ib_df_percentile = ib_df.loc[ib_df.ib_yield > ib_df.ib_yield.quantile(percentile)]
    else:
        ib_df_percentile = ib_df.copy(deep=False)
        
    ib_corr_returns = ib_df.ib_yield.corr(ib_df.Close)
    
    return np.mean(ib_df_percentile.Close), ib_corr_returns, ib_df_percentile

In [56]:
earn_mean,earn_corr,earn_df = earnings_yield(data_combined, df_fundamentals, percentile = 0.90)

In [57]:
earn_mean

0.15431902676585418

In [58]:
earn_corr

-0.009397600208458236